In [2]:
import random  # Rastgele sayı üretimi için (Rulet, Mutasyon vb.)
import math    # Karekök (sqrt) hesabı için (Mesafe ölçümü)
import time    # Kodun ne kadar sürdüğünü ölçmek için
import pandas as pd # Verileri Excel'e kaydetmek için kullanılan kütüphane
import copy    # Listeleri kopyalamak gerekirse diye (Güvenlik)

# =============================================================================
# 1. VERİ SETLERİ (Şehir Koordinatları)
# =============================================================================

# Berlin52 Veri Seti: (x, y) koordinatları
berlin52_coords = [
    (565, 575), (25, 185), (345, 750), (945, 685), (845, 655), (880, 660), (25, 230),
    (525, 1000), (580, 1175), (650, 1130), (1605, 620), (1220, 580), (1465, 200), (1530, 5),
    (845, 680), (725, 370), (145, 665), (415, 635), (510, 875), (560, 365), (300, 465),
    (520, 585), (480, 415), (835, 625), (975, 580), (1215, 245), (1320, 315), (1250, 400),
    (660, 180), (410, 250), (420, 555), (575, 665), (1150, 1160), (700, 580), (685, 595),
    (685, 610), (770, 610), (795, 645), (720, 635), (760, 650), (475, 960), (95, 260),
    (875, 920), (700, 500), (555, 815), (830, 485), (1170, 65), (830, 610), (605, 625),
    (595, 360), (1340, 725), (1740, 245)
]

# Eil51 Veri Seti
eil51_coords = [
    (37, 52), (49, 49), (52, 64), (20, 26), (40, 30), (21, 47), (17, 63), (31, 62),
    (52, 33), (51, 21), (42, 41), (31, 32), (5, 25), (12, 42), (36, 16), (52, 41),
    (27, 23), (17, 33), (13, 13), (57, 58), (62, 42), (42, 57), (16, 57), (8, 52),
    (7, 38), (27, 68), (30, 48), (43, 67), (58, 48), (58, 27), (37, 69), (38, 46),
    (46, 10), (61, 33), (62, 63), (63, 69), (32, 22), (45, 35), (59, 15), (5, 6),
    (10, 17), (21, 10), (5, 64), (30, 15), (39, 10), (32, 39), (25, 32), (25, 55),
    (48, 28), (56, 37), (30, 40)
]

# =============================================================================
# 2. YARDIMCI VE GENETİK OPERATÖRLER
# =============================================================================

# Fonksiyon: Bir rotanın toplam maliyetini (mesafesini) hesaplar
def calc_cost(route, coords):
    cost = 0  # Toplam maliyeti tutacak değişken
    N = len(route)  # Şehir sayısı
    for i in range(N):  # Her şehri gez
        # Şu anki şehir (p1) ve bir sonraki şehir (p2)
        # (i+1)%N yapıyoruz ki son şehirden ilk şehre (döngü) geri dönebilsin
        p1, p2 = coords[route[i]], coords[route[(i+1)%N]]
        # Öklid mesafesi formülü: sqrt((x1-x2)^2 + (y1-y2)^2)
        cost += math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
    return cost  # Toplam mesafeyi döndür

# Fonksiyon: Başlangıç popülasyonunu oluşturur
def create_pop(size, num_cities):
    # size kadar birey üret, her biri şehirlerin rastgele sıralanmış halidir
    # random.sample: Bir listeyi karıştırır (shuffle eder)
    return [random.sample(range(num_cities), num_cities) for _ in range(size)]

# --- SEÇİM (SELECTION) YÖNTEMLERİ ---

# Turnuva Seçimi: Rastgele k kişi seç, en iyisini al
def sel_tournament(pop, costs, k=3):
    # Popülasyondan rastgele k tane indeks seç (Örn: 3 kişi)
    idxs = random.sample(range(len(pop)), k)
    # Bu indeksler arasından maliyeti (costs[i]) en düşük olanı bul
    best = min(idxs, key=lambda i: costs[i])
    return pop[best]  # Kazanan bireyi döndür

# Rulet Tekerleği Seçimi: İyilerin şansı yüksek, kötülerin düşük
def sel_roulette(pop, costs):
    # Fitness hesabı: Maliyet ne kadar azsa, Fitness (1/Cost) o kadar büyük olur
    fitness = [1/c for c in costs]
    total = sum(fitness)  # Toplam fitness (Pastanın tamamı)
    r = random.uniform(0, total)  # 0 ile Toplam arasında rastgele bir ibre at
    upto = 0  # Kümülatif toplam
    for i, f in enumerate(fitness):
        upto += f  # Dilimi ekle
        if r <= upto: return pop[i]  # İbre bu dilime denk geldiyse seç
    return pop[-1]  # Matematiksel hata olursa sonuncuyu döndür

# --- ÇAPRAZLAMA (CROSSOVER) YÖNTEMLERİ ---

# PMX (Partially Mapped Crossover) - Kısmi Haritalamalı
def cross_pmx(p1, p2):
    N = len(p1)
    # İki kesim noktası (a ve b) belirle
    a, b = sorted(random.sample(range(N), 2))
    child = [-1]*N  # İçi boş (-1) bir çocuk yarat
    child[a:b] = p1[a:b]  # Anne'nin (p1) parçasını çocuğa kopyala

    # Kalan boşlukları doldur
    for i in range(N):
        if i >= a and i < b: continue  # Zaten dolu olan aralığı atla
        val = p2[i]  # Baba'daki sayıyı al
        # Eğer bu sayı Anne'den aldığımız parçada zaten varsa (Çakışma!)
        while val in p1[a:b]:
            # Haritalama yap: p1'deki yerini bul, p2'de o yere karşılık geleni al
            val = p2[p1.index(val)]
        child[i] = val  # Çakışmayan sayıyı yerleştir
    return child

# OX (Order Crossover) - Sıralı Çaprazlama
def cross_ox(p1, p2):
    N = len(p1)
    a, b = sorted(random.sample(range(N), 2))
    child = [-1]*N
    child[a:b] = p1[a:b]  # Anne'den parçayı kopyala

    present = set(child[a:b])  # Çocukta var olan sayıları kaydet
    ptr = 0  # Baba listesi üzerinde gezecek işaretçi
    for i in range(N):
        if child[i] == -1:  # Eğer çocukta burası boşsa
            # Baba'da sıradaki sayı çocukta zaten var mı diye bak
            while p2[ptr] in present: ptr += 1  # Varsa geç
            child[i] = p2[ptr]  # Yoksa yerleştir
            ptr += 1
    return child

# --- MUTASYON VE STRATEJİLER ---

# Inversion Mutation: Bir aralığı ters çevirir (Düğümleri çözer)
def mut_inversion(ind, rate):
    if random.random() < rate:  # Olasılık tutarsa
        i, j = sorted(random.sample(range(len(ind)), 2))  # İki nokta seç
        ind[i:j+1] = ind[i:j+1][::-1]  # Aralığı tersine çevir (Reverse)
    return ind

# Swap Mutation: İki şehri yer değiştirir (Basit karıştırma)
def mut_swap(ind, rate):
    if random.random() < rate:  # Olasılık tutarsa
        i, j = random.sample(range(len(ind)), 2)  # İki nokta seç
        ind[i], ind[j] = ind[j], ind[i]  # Yer değiştir
    return ind

# 2-Opt Local Search: Rotayı iyileştiren "Cila" işlemi
def imp_2opt(route, coords):
    best_r = route[:]  # Mevcut rotayı kopyala
    best_c = calc_cost(route, coords)  # Maliyetini hesapla
    N = len(route)
    # Tüm kenar ikililerini dene (i ve j)
    for i in range(1, N-1):
        for j in range(i+1, N):
            if j-i == 1: continue  # Yan yana olanları atla
            # Yeni bir rota dene: i ile j arasını ters çevir
            new_r = best_r[:]
            new_r[i:j] = best_r[i:j][::-1]
            # Eğer yeni rota daha kısaysa
            if calc_cost(new_r, coords) < best_c:
                return new_r  # İyileştirilmiş rotayı hemen döndür (Hız için)
    return best_r  # İyileşme yoksa eskisini döndür

# =============================================================================
# 3. EVRENSEL GENETİK MOTOR (Her Şeyi Çalıştıran Fonksiyon)
# =============================================================================

def run_ga_universal(dataset_name, coords, optimum,
                     sel_name, cross_name, surv_name, strat_name,
                     pop_size, mut_rate, run_id):

    # Survival Tipi'ne göre çocuk üretim oranını (Substitution Rate) belirle
    if surv_name == "Generational":
        sub_rate = 1.0  # Nüfusun tamamı (%100) değişecek
    else:
        sub_rate = 0.5  # SteadyState: Nüfusun yarısı (%50) değişecek

    max_gens = 150  # Maksimum kaç nesil dönecek
    patience = 20   # Kaç nesil gelişme olmazsa dursun (Sabır)

    start_t = time.time()  # Süreyi başlat
    N = len(coords)  # Şehir sayısı
    pop = create_pop(pop_size, N)  # Rastgele ilk popülasyonu yarat

    global_best = float('inf')  # En iyi skoru sonsuz yap (ki ilki onu geçsin)
    stag_count = 0  # Gelişme olmayan nesil sayacı
    stop_gen = 0  # Hangi nesilde durdu

    # --- NESİL DÖNGÜSÜ (EVOLUSYON) ---
    for g in range(1, max_gens+1):

        # [STRATEJİ] Eğer yöntem 2-Opt ise, nüfusun bir kısmına cila çek
        if strat_name == "2Opt":
             for i in range(len(pop)):
                 if random.random() < 0.05: # %5 şansla bireye 2-Opt uygula
                     pop[i] = imp_2opt(pop[i], coords)

        # Tüm bireylerin maliyetini hesapla
        costs = [calc_cost(p, coords) for p in pop]
        best_now = min(costs)  # Bu neslin en iyisini bul

        # [DURDURMA KRİTERİ] Kontrolü
        if best_now < global_best:  # Rekor kırıldı mı?
            global_best = best_now  # Yeni rekoru kaydet
            stag_count = 0          # Sayacı sıfırla (Sabrımız tazelendi)
        else:
            stag_count += 1         # Rekor yok, sabrımız azalıyor

        if stag_count >= patience:  # Sabır taştıysa
            stop_gen = g
            break  # Döngüyü kır, bitir.
        stop_gen = g

        # [ÜREME] Yeni çocukların üretilmesi
        target_kids = int(pop_size * sub_rate)  # Hedef çocuk sayısı
        kids = []

        while len(kids) < target_kids:
            # 1. EBEVEYN SEÇİMİ (Selection)
            if sel_name == "Rulet":
                p1, p2 = sel_roulette(pop, costs), sel_roulette(pop, costs)
            else: # Turnuva
                p1, p2 = sel_tournament(pop, costs), sel_tournament(pop, costs)

            # 2. ÇAPRAZLAMA (Crossover)
            if cross_name == "PMX":
                c = cross_pmx(p1, p2)
            else: # OX
                c = cross_ox(p1, p2)

            # 3. MUTASYON (Mutation)
            if strat_name == "Inversion":
                c = mut_inversion(c, mut_rate) # Inversion uygula
            else:
                # 2-Opt stratejisinde temel mutasyon swap'tır
                c = mut_swap(c, mut_rate)

            kids.append(c)  # Çocuğu listeye ekle

        # [SURVIVAL] Popülasyon güncelleme (SORUNUN CEVABI BURADA)
        # Mevcut popülasyonu maliyete göre sırala (Azdan çoka / İyiden Kötüye)
        zipped = sorted(list(zip(pop, costs)), key=lambda x: x[1])

        if surv_name == "Generational":
            # Generational: Tüm nesil silinir
            new_pop = [zipped[0][0]] # Ama en iyi 1 kişi (Elit) korunur
            # Çocukları ekle
            for k in kids:
                if len(new_pop) < pop_size: new_pop.append(k)
            # Eğer eksik kaldıysa (matematiksel küsurattan), eskilerden tamamla
            idx = 1
            while len(new_pop) < pop_size:
                new_pop.append(zipped[idx][0])
                idx += 1
            pop = new_pop # Popülasyonu güncelle

        else: # SteadyState (Kötüleri At)
            # Kaç kişi kalacak? (Örn: 100 - 50 = 50 kişi)
            keep = pop_size - target_kids

            # Listeyi dilimle (Slicing):
            # zipped[:keep] -> Listenin BAŞINDAN 'keep' kadarını al.
            # Liste 'İyiden Kötüye' sıralı olduğu için, EN İYİ %50 alınır.
            # Otomatikman EN KÖTÜ %50 dışarıda kalır (Elenir).
            pop = [z[0] for z in zipped[:keep]] + kids

    # Bitiş işlemleri
    duration = time.time() - start_t
    # Optimuma ne kadar uzak kaldık? (Sapma oranı)
    gap = (global_best - optimum) / optimum * 100

    # Sonuçları sözlük olarak döndür
    return {
        "best_cost": global_best,
        "gap": gap,
        "time": duration,
        "stop_gen": stop_gen
    }

# =============================================================================
# 4. ORCHESTRATOR (Deney Yönetici)
# =============================================================================

def main():
    print("="*100)
    print("   UNIVERSAL GENETIC ALGORITHM BENCHMARK (640 RUNS)")
    print("   Uyarı: Bu işlem donanım hızına bağlı olarak zaman alabilir.")
    print("="*100)

    datasets = [("Berlin52", berlin52_coords, 7542), ("Eil51", eil51_coords, 426)]

    # TEST EDİLECEK PARAMETRELER (Full Factorial Design)
    selections = ["Rulet", "Turnuva"]
    crossovers = ["PMX", "OX"]
    survivals  = ["Generational", "SteadyState"]
    strategies = ["Inversion", "2Opt"]
    pop_sizes  = [50, 100]
    mut_rates  = [0.01, 0.10]

    excel_data = [] # Verileri tutacak liste
    run_counter = 0 # Toplam kaçıncı deneydeyiz
    total_runs = 2 * 2 * 2 * 2 * 2 * 2 * 2 * 5 # Toplam 640

    start_all = time.time() # Toplam süreyi başlat

    # İÇ İÇE DÖNGÜLER (Her parametre diğerleriyle çarpışır)
    for d_name, d_coords, d_opt in datasets:
        print(f"\n >>> DATASET: {d_name} BAŞLIYOR...")

        for sel in selections:
            for cross in crossovers:
                for surv in survivals:
                    for strat in strategies:
                        for pop in pop_sizes:
                            for mut in mut_rates:

                                # Her kombinasyon için 5 tekrar (Run)
                                for r in range(1, 6):
                                    run_counter += 1

                                    # Ekrana bilgi bas (Tek satırda güncelleyerek)
                                    print(f"\r[{run_counter}/{total_runs}] {d_name}|{sel}|{cross}|{surv}|{strat}|Pop:{pop}|Mut:{mut} -> Çalışıyor...", end="")

                                    # Deneyi çalıştır
                                    res = run_ga_universal(
                                        d_name, d_coords, d_opt,
                                        sel, cross, surv, strat,
                                        pop, mut, r
                                    )

                                    # Sonucu listeye ekle
                                    excel_data.append({
                                        "Dataset": d_name,
                                        "Selection": sel,
                                        "Crossover": cross,
                                        "Survival": surv,
                                        "Strategy": strat,
                                        "Pop_Size": pop,
                                        "Mut_Rate": mut,
                                        "Run_ID": r,
                                        "Best_Cost": res["best_cost"],
                                        "Gap_Percent": res["gap"],
                                        "Time_Sec": res["time"],
                                        "Stop_Gen": res["stop_gen"]
                                    })
        print(f"\n   -> {d_name} tamamlandı.")

    total_time = time.time() - start_all

    # EXCEL KAYIT İŞLEMİ
    print("\n" + "="*80)
    print(f"   TÜM İŞLEMLER BİTTİ ({total_time:.1f} saniye).")
    print("   'ga_universal_benchmark.xlsx' dosyasına yazılıyor...")

    # Veriyi Pandas DataFrame'e çevir
    df = pd.DataFrame(excel_data)

    # Sütun sırasını düzenli hale getir
    cols = ["Dataset", "Selection", "Crossover", "Survival", "Strategy", "Pop_Size", "Mut_Rate",
            "Run_ID", "Best_Cost", "Gap_Percent", "Time_Sec", "Stop_Gen"]
    df = df[cols]

    try:
        df.to_excel("ga_universal_benchmark.xlsx", index=False)
        print("   >>> BAŞARILI: Dosya oluşturuldu.")
    except Exception as e:
        print(f"   >>> HATA: Dosya kaydedilemedi. ({e})")

# Bu dosya direkt çalıştırılırsa main() fonksiyonunu çağır
if __name__ == "__main__":
    main()

   UNIVERSAL GENETIC ALGORITHM BENCHMARK (640 RUNS)
   Uyarı: Bu işlem donanım hızına bağlı olarak zaman alabilir.

 >>> DATASET: Berlin52 BAŞLIYOR...
[320/640] Berlin52|Turnuva|OX|SteadyState|2Opt|Pop:100|Mut:0.1 -> Çalışıyor...
   -> Berlin52 tamamlandı.

 >>> DATASET: Eil51 BAŞLIYOR...
[640/640] Eil51|Turnuva|OX|SteadyState|2Opt|Pop:100|Mut:0.1 -> Çalışıyor...
   -> Eil51 tamamlandı.

   TÜM İŞLEMLER BİTTİ (508.3 saniye).
   'ga_universal_benchmark.xlsx' dosyasına yazılıyor...
   >>> BAŞARILI: Dosya oluşturuldu.
